In [1]:
from pyomo.environ import*

Definir modelo:

In [2]:
model = ConcreteModel()

Definimos datos:
model.V = Numero total de vueltas.
model.Va = Set utilizado para relacionar la vuelta i + 1 con la vuelta i.
Tpits = Tiempo que se pierde en una parada en pits. (s)
To = Tiempo de vuelta para neumáticos nuevos.(s)

In [3]:
model.V = Set (initialize = range(60))
model.Va = Set (initialize = range(59))
Tpits = 22
To = 90.414


Definimos variables:
T = Tiempo de cada vuelta (s).
v = Numero de vueltas consecutivas sin parar en pits.
x = Condición de parada en pits de cada vuelta. 

In [4]:
model.T = Var (model.V, initialize = 70 , within = NonNegativeReals)
model.v = Var (model.V, within = NonNegativeIntegers)
model.x = Var (model.V, within = Binary, initialize = 0)



Definimos objetivo: Minimizar el tiempo total de carrera comprendido por la suma del tiempo de cada vuelta y el de las paradas en boxes

In [5]:
model.Obj = Objective (expr = sum(model.T[i]for i in model.V) + sum(model.x[i]*Tpits for i in model.V))

Insertamos Restricciones: 
Ti = Tiempo de cada vuelta según las vueltas que han pasado desde la última parada.
Vi = Contador de las vueltas que han pasado desde la última parada.
Vo = Inicialización del contador de vueltas

In [6]:
model.Ti = ConstraintList()
for i in model.v:
    model.Ti.add(
        model.T[i] == 0.005 * (model.v[i])**2 - 0.0243 * model.v[i] + To
    )
model.Vi = ConstraintList()
for i in model.Va:
    model.Vi.add(
        model.v[i+1] == (model.v[i] + 1) *(1- model.x[i])
    )

model.Vo = Constraint (expr = model.v[0] == 0 )



Se resuelve el modelo y los Xi que sean 1 son las vueltas en las que hay que parar:

In [7]:
results = SolverFactory ('couenne',tee=True).solve(model)
results.write()
model.pprint()
print('*** Solution *** :')
print('El tiempo total es:', value(model.Obj), 'segundos, es decir,', value(model.Obj) //3600, 'horas', value(model.Obj) //60 - 60*(value(model.Obj) //3600), 'minutos', value(model.Obj)//1 -60*(value(model.Obj) //60), 'segundos' )

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 180
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: couenne\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 6.3640289306640625
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
4 Set Declarations
    Ti_i